In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import zipfile
import os
import pandas as pd

# Path to the zip file
zip_file_path = '/content/drive/MyDrive/environment recognition/places365_contents.zip'

# Directory to extract the contents
extracted_dir = 'data_contents/'

# Extract the contents of the zip file
with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
    zip_ref.extractall(extracted_dir)

# Create a list to store dataframe rows
data = []

# Traverse through the extracted directory to create the dataframe
for root, dirs, files in os.walk(extracted_dir):
    for file in files:
        if file.endswith('.jpg'):
            file_path = os.path.join(root, file)
            label_name = os.path.basename(root)
            data.append({'path': file_path, 'name': file, 'label': label_name})

# Create a DataFrame
df = pd.DataFrame(data)

# Adjust path format for 'path name' column
df['path name'] = 'data_contents/' + df['label'] + '/' + df['name']

# Rearrange columns
df = df[['path name', 'name', 'label']]

# Print the DataFrame
print(df)

                                       path name          name           label
0      data_contents/airplane_cabin/00002752.jpg  00002752.jpg  airplane_cabin
1      data_contents/airplane_cabin/00002732.jpg  00002732.jpg  airplane_cabin
2      data_contents/airplane_cabin/00003353.jpg  00003353.jpg  airplane_cabin
3      data_contents/airplane_cabin/00002966.jpg  00002966.jpg  airplane_cabin
4      data_contents/airplane_cabin/00003053.jpg  00003053.jpg  airplane_cabin
...                                          ...           ...             ...
62515   data_contents/garage-indoor/00002366.jpg  00002366.jpg   garage-indoor
62516   data_contents/garage-indoor/00003245.jpg  00003245.jpg   garage-indoor
62517   data_contents/garage-indoor/00004236.jpg  00004236.jpg   garage-indoor
62518   data_contents/garage-indoor/00002776.jpg  00002776.jpg   garage-indoor
62519   data_contents/garage-indoor/00004939.jpg  00004939.jpg   garage-indoor

[62520 rows x 3 columns]


In [7]:
import tensorflow as tf
from tensorflow.keras.layers import Conv2D, Dense, Flatten, Reshape
from tensorflow.keras.models import Sequential

# Define the squash activation function for capsules
def squash(x, axis=-1):
    norm = tf.norm(x, axis=axis, keepdims=True)
    return (norm / (1 + norm**2)) * x

# Create the Capsule Layer
class CapsuleLayer(tf.keras.layers.Layer):
    def __init__(self, num_capsules, capsule_dim, num_routing=3, **kwargs):
        super(CapsuleLayer, self).__init__(**kwargs)
        self.num_capsules = num_capsules
        self.capsule_dim = capsule_dim
        self.num_routing = num_routing

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.kernel = self.add_weight(
            'kernel',
            shape=[input_dim, self.num_capsules * self.capsule_dim],
            initializer='glorot_uniform',
            trainable=True
        )
        super(CapsuleLayer, self).build(input_shape)

    def call(self, inputs):
        # Transform the input capsules using the weight matrix
        inputs = tf.reshape(inputs, [-1, inputs.shape[-1]])
        caps_output = tf.matmul(inputs, self.kernel)

        # Reshape into capsules
        caps_output = tf.reshape(caps_output, [-1, self.num_capsules, self.capsule_dim])

        # Dynamic routing algorithm
        # Implement the routing algorithm here

        # Squash the output capsules
        caps_output = squash(caps_output)
        return caps_output

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.num_capsules, self.capsule_dim)

# Create a simple Capsule Network model
def create_capsule_model(input_shape, num_classes):
    model = Sequential([
        Conv2D(64, (3, 3), padding='valid', input_shape=input_shape, activation='relu'),
        Conv2D(128, (3, 3), padding='valid', activation='relu'),
        CapsuleLayer(num_capsules=10, capsule_dim=16),  # You can adjust these parameters
        Flatten(),
        Dense(num_classes, activation='softmax')
    ])
    return model

# Define input shape and number of classes
input_shape = (112, 112, 3)  # Change this according to your data
num_classes = 13  # Change this to the correct number of classes

# Create the Capsule Network model
model = Sequential([
    Conv2D(64, (3, 3), padding='valid', input_shape=input_shape, activation='relu'),
    Conv2D(128, (3, 3), padding='valid', activation='relu'),
    CapsuleLayer(num_capsules=13, capsule_dim=16),  # Change the number of capsules
    Flatten(),
    Dense(num_classes, activation='softmax')
])

# Print the model summary
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 110, 110, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 108, 108, 128)     73856     
                                                                 
 capsule_layer (CapsuleLaye  (None, 13, 16)            26624     
 r)                                                              
                                                                 
 flatten (Flatten)           (None, 208)               0         
                                                                 
 dense (Dense)               (None, 13)                2717      
                                                                 
Total params: 104989 (410.11 KB)
Trainable params: 104989 (410.11 KB)
Non-trainable params: 0 (0.00 Byte)
________________

In [4]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split

# Assuming your DataFrame is named 'df'
# Split the DataFrame into train and validation sets
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# ImageDataGenerator for data augmentation and preprocessing
datagen = ImageDataGenerator(
    rescale=1.0/255,  # Rescale pixel values to [0, 1]
    rotation_range=20,  # Data augmentation: random rotations
    width_shift_range=0.2,  # Data augmentation: random width shifts
    height_shift_range=0.2,  # Data augmentation: random height shifts
    horizontal_flip=True,  # Data augmentation: horizontal flips
)
batch_size = 32
# Load and preprocess training data from DataFrame
train_generator = datagen.flow_from_dataframe(
    train_df,
    x_col='path name',  # Column containing image paths
    y_col='label',  # Column containing labels
    target_size=(112, 112),  # Resize images to match model input size
    batch_size=batch_size,
    class_mode='categorical',  # Categorical labels for multi-class classification
)

# Load and preprocess validation data from DataFrame
validation_generator = datagen.flow_from_dataframe(
    val_df,
    x_col='path name',
    y_col='label',
    target_size=(112, 112),
    batch_size=batch_size,
    class_mode='categorical',
)

Found 50016 validated image filenames belonging to 13 classes.
Found 12504 validated image filenames belonging to 13 classes.


In [5]:
import tensorflow as tf
from tensorflow.keras.models import load_model

model = load_model('/content/drive/MyDrive/environment recognition/capsnet_model_100_epochs.h5')

In [6]:
# Evaluate the model on validation data
val_loss, val_acc = model.evaluate(validation_generator)

print("Validation Loss:", val_loss)
print("Validation Accuracy:", val_acc)

# # Save the model
# model.save('/content/drive/MyDrive/envirnment recognition/model_200_epochs.h5')  # Adjust the filename as needed

391/391 [==============================] - 121s 307ms/step - loss: 11.9517 - accuracy: 0.1508
Validation Loss: 11.951667785644531
Validation Accuracy: 0.1507517546415329
